<a href="https://colab.research.google.com/github/nonyeezeh/Research-Project-Code/blob/main/NN_Sparse_1_10_Relu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [6]:
pip install pgmpy

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
from tabulate import tabulate

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers, callbacks, regularizers

from scipy.stats import entropy

# Bayesian Network Data Generation 1000, 2000, ..., 10000 Samples (sparse)

In [4]:
# Function to generate CPDs for a Sparse BN with random relationships
def generate_sparse_cpds():
    # Generate random probabilities for IR
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()  # Normalize to make it a valid probability distribution

    # Generate random probabilities for EI
    ei_probs = np.random.rand(3)
    ei_probs /= ei_probs.sum()  # Normalize to make it a valid probability distribution

    # Generate random probabilities for SP, either given IR, EI, or both, randomly chosen
    relationship = np.random.choice(['IR', 'EI', 'IR_EI'])

    if relationship == 'IR':
        # SP depends only on IR
        sp_probs = np.random.rand(3, 3)
        sp_probs /= sp_probs.sum(axis=0, keepdims=True)
        return ir_probs, ei_probs, sp_probs, 'IR'

    elif relationship == 'EI':
        # SP depends only on EI
        sp_probs = np.random.rand(3, 3)
        sp_probs /= sp_probs.sum(axis=0, keepdims=True)
        return ir_probs, ei_probs, sp_probs, 'EI'

    else:
        # SP depends on both IR and EI
        sp_probs = np.random.rand(3, 3, 3)
        sp_probs /= sp_probs.sum(axis=0, keepdims=True)
        return ir_probs, ei_probs, sp_probs, 'IR_EI'

# Function to generate and save samples for Sparse BN
def generate_and_save_sparse_samples(ir_probs, ei_probs, sp_probs, relationship, sample_size, filename):
    output_data = []

    # Generate `sample_size` random samples
    for _ in range(sample_size):
        # Sample `IR` state based on `IR` probabilities
        ir_state_idx = np.random.choice(3, p=ir_probs)
        ir_state = ['low', 'medium', 'high'][ir_state_idx]
        ir_prob = ir_probs[ir_state_idx]

        # Sample `EI` state based on `EI` probabilities
        ei_state_idx = np.random.choice(3, p=ei_probs)
        ei_state = ['poor', 'average', 'good'][ei_state_idx]
        ei_prob = ei_probs[ei_state_idx]

        # Sample `SP` state based on the random relationship
        if relationship == 'IR':
            # SP depends only on IR
            sp_probs_given_ir = sp_probs[:, ir_state_idx]
            sp_state_idx = np.random.choice(3, p=sp_probs_given_ir)
            sp_state = ['decrease', 'stable', 'increase'][sp_state_idx]
            sp_prob = sp_probs_given_ir[sp_state_idx]

        elif relationship == 'EI':
            # SP depends only on EI
            sp_probs_given_ei = sp_probs[:, ei_state_idx]
            sp_state_idx = np.random.choice(3, p=sp_probs_given_ei)
            sp_state = ['decrease', 'stable', 'increase'][sp_state_idx]
            sp_prob = sp_probs_given_ei[sp_state_idx]

        else:
            # SP depends on both IR and EI
            sp_probs_given_ir_ei = sp_probs[:, ir_state_idx, ei_state_idx]
            sp_state_idx = np.random.choice(3, p=sp_probs_given_ir_ei)
            sp_state = ['decrease', 'stable', 'increase'][sp_state_idx]
            sp_prob = sp_probs_given_ir_ei[sp_state_idx]

        # Append sample data to output list
        output_data.append({
            'IR_State': ir_state,
            'IR_Prob': f'{ir_prob:.4f}',
            'EI_State': ei_state,
            'EI_Prob': f'{ei_prob:.4f}',
            'SP_Probabilities (decrease, stable, increase)': ', '.join([f'{prob:.4f}' for prob in (
              sp_probs_given_ir_ei if relationship == 'IR_EI' else
              sp_probs_given_ir if relationship == 'IR' else
              sp_probs_given_ei
            )]),
            'Chosen_SP_State': sp_state,
            'Chosen_SP_Probability': f'{sp_prob:.4f}',
            'Relationship': relationship
        })

    # Create a DataFrame from the output data
    output_df = pd.DataFrame(output_data)

    # Save the output DataFrame to a CSV file
    output_df.to_csv(filename, index=False)

    # Print the first few rows for visual confirmation
    print(f"\nSample size: {sample_size} - First few rows of generated samples:\n")
    print(tabulate(output_df.head(), headers='keys', tablefmt='grid'))

# Generate and save samples for sample sizes from 1000 to 10000 every 1000 for Sparse BN
sample_sizes = range(1000, 11000, 1000)

for size in sample_sizes:
    # Generate the CPDs for Sparse BN
    ir_probs, ei_probs, sp_probs, relationship = generate_sparse_cpds()

    # Generate and save individual samples for the given sample size
    generate_and_save_sparse_samples(ir_probs, ei_probs, sp_probs, relationship, size, f'combined_sparse_probabilities_{size}.csv')

# Notify the user that the process is done
print("\nGeneration and saving of individual samples complete for all sample sizes (Sparse BN)!")


Sample size: 1000 - First few rows of generated samples:

+----+------------+-----------+------------+-----------+-------------------------------------------------+-------------------+-------------------------+----------------+
|    | IR_State   |   IR_Prob | EI_State   |   EI_Prob | SP_Probabilities (decrease, stable, increase)   | Chosen_SP_State   |   Chosen_SP_Probability | Relationship   |
+====+============+===========+============+===========+=================================================+===================+=========================+================+
|  0 | low        |    0.2781 | good       |    0.4104 | 0.2038, 0.0124, 0.7838                          | increase          |                  0.7838 | EI             |
+----+------------+-----------+------------+-----------+-------------------------------------------------+-------------------+-------------------------+----------------+
|  1 | high       |    0.3065 | poor       |    0.5134 | 0.2999, 0.4257, 0.2745            

# Hypothesis Model 1000, 2000, ..., 10000 Samples (sparse) 1 hidden Layer, 10 Neurons Relu

In [11]:
# Sample sizes to loop through
sample_sizes = range(1000, 11000, 1000)

# Define the Neural Network architecture with L2 regularization
def create_nn_model(input_shape, hidden_layers=1, nodes_per_layer=10, l2_lambda=0.01):
    model = models.Sequential()

    # Input layer with dynamic input shape (1 or 2 features)
    model.add(layers.InputLayer(input_shape=(input_shape,)))

    # Hidden layers with L2 regularization and Dropout
    for layer_num in range(hidden_layers):
        model.add(layers.Dense(
            nodes_per_layer,
            activation='relu',
            kernel_regularizer=regularizers.l2(l2_lambda),  # L2 regularization
            name=f"hidden_layer_{layer_num + 1}"
        ))
        model.add(layers.Dropout(0.2))  # Dropout layer to reduce overfitting

    # Output layer (3 classes: decrease, stable, increase) with L2 regularization
    model.add(layers.Dense(
        3,
        activation='softmax',
        kernel_regularizer=regularizers.l2(l2_lambda),  # L2 regularization
        name="output_layer"
    ))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Prepare a dictionary to store the extracted data for each sample size
extracted_data = {}

# Extract the required columns from all sample sizes first
for size in sample_sizes:
    # Load data for the current sample size (adjust the file paths if necessary)
    outcomes_file = f'combined_sparse_probabilities_{size}.csv'
    df = pd.read_csv(outcomes_file)

    # Extract columns based on relationship
    relationship = df['Relationship'][0]  # Assuming all rows have the same relationship in this sample

    # Determine which columns to extract based on relationship
    if relationship == 'IR EI':
        required_columns = ['IR_State', 'EI_State', 'Chosen_SP_State']
    elif relationship == 'IR':
        required_columns = ['IR_State', 'Chosen_SP_State']
    elif relationship == 'EI':
        required_columns = ['EI_State', 'Chosen_SP_State']

    df_extracted = df[required_columns]

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    if 'IR_State' in df_extracted.columns:
        df_extracted['IR_encoded'] = df_extracted['IR_State'].map(ir_map)
    if 'EI_State' in df_extracted.columns:
        df_extracted['EI_encoded'] = df_extracted['EI_State'].map(ei_map)
    df_extracted['SP_encoded'] = df_extracted['Chosen_SP_State'].map(sp_map)

    # Store the extracted and encoded data for later use
    extracted_data[size] = df_extracted

# Loop through each sample size for NN training, validation, and testing
for size in sample_sizes:
    # Retrieve the extracted data for the current sample size
    df = extracted_data[size]

    # Determine features (X) and labels (y) based on available columns
    if 'IR_encoded' in df.columns and 'EI_encoded' in df.columns:
        X = df[['IR_encoded', 'EI_encoded']]
    elif 'IR_encoded' in df.columns:
        X = df[['IR_encoded']]
    elif 'EI_encoded' in df.columns:
        X = df[['EI_encoded']]

    y = df['SP_encoded']

    # Refresh the data split for each iteration
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False, random_state=42)

    # Show split confirmation
    print(f"\nSample size: {size}")
    print("Training Data:", X_train.shape, y_train.shape)
    print("Validation Data:", X_val.shape, y_val.shape)
    print("Test Data:", X_test.shape, y_test.shape)

    # Create the Neural Network model with L2 regularization
    input_shape = X_train.shape[1]  # Number of features (1 or 2)
    nn_model = create_nn_model(input_shape=input_shape, hidden_layers=1, nodes_per_layer=10, l2_lambda=0.01)

    # Early stopping callback to prevent overfitting
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model
    history = nn_model.fit(X_train, y_train,
                           epochs=50,
                           batch_size=32,
                           validation_data=(X_val, y_val),
                           callbacks=[early_stopping],
                           verbose=0)  # Set verbose=0 to avoid too much output

    # Print training, validation, and test accuracy
    train_loss, train_accuracy = nn_model.evaluate(X_train, y_train, verbose=0)
    val_loss, val_accuracy = nn_model.evaluate(X_val, y_val, verbose=0)
    test_loss, test_accuracy = nn_model.evaluate(X_test, y_test, verbose=0)
    print(f"Training Accuracy for {size} samples: {train_accuracy:.4f}")
    print(f"Validation Accuracy for {size} samples: {val_accuracy:.4f}")
    print(f"Test Accuracy for {size} samples: {test_accuracy:.4f}")

    # Make predictions on the test set
    predictions = nn_model.predict(X_test)

    # Convert the predicted probabilities to class labels
    predicted_classes = predictions.argmax(axis=1)

    # Create a list to map integers back to the original SP labels
    sp_reverse_map = ['decrease', 'stable', 'increase']

    # Convert the predicted classes to the original labels
    predicted_labels = [sp_reverse_map[label] for label in predicted_classes]

    # Create a DataFrame for the predicted probabilities
    probs_df = pd.DataFrame(predictions, columns=['Prob_decrease', 'Prob_stable', 'Prob_increase'])

    # Output the features, predicted SP, and the NN probabilities
    result_df = X_test.copy()
    result_df['Predicted_SP'] = predicted_labels

    # Combine the result with the predicted probabilities
    combined_df = pd.concat([result_df.reset_index(drop=True), probs_df.reset_index(drop=True)], axis=1)

    # Save the test data with predictions to a CSV file
    combined_df.to_csv(f'test_data_nn_sparse_{size}.csv', index=False)

    # Show the first few rows of the results for this sample size
    print(f"\nPredicted Results and Probabilities for {size} samples (First 15 rows):")
    print(combined_df.head(15))

# After the loop is done, print this message
print("\nLooping through all sample sizes complete!")

<ipython-input-11-7bea497a529a>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extracted['EI_encoded'] = df_extracted['EI_State'].map(ei_map)
<ipython-input-11-7bea497a529a>:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extracted['SP_encoded'] = df_extracted['Chosen_SP_State'].map(sp_map)
<ipython-input-11-7bea497a529a>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d


Sample size: 1000
Training Data: (700, 1) (700,)
Validation Data: (150, 1) (150,)
Test Data: (150, 1) (150,)
Training Accuracy for 1000 samples: 0.5657
Validation Accuracy for 1000 samples: 0.6000
Test Accuracy for 1000 samples: 0.6200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 

Predicted Results and Probabilities for 1000 samples (First 15 rows):
    EI_encoded Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0            2     increase       0.227285     0.058955       0.713760
1            2     increase       0.227285     0.058955       0.713760
2            0       stable       0.315123     0.343918       0.340959
3            2     increase       0.227285     0.058955       0.713760
4            2     increase       0.227285     0.058955       0.713760
5            0       stable       0.315123     0.343918       0.340959
6            0       stable       0.315123     0.343918       0.340959
7            0       stable       0.315123     0.343918       0.340959
8            0 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Training Accuracy for 2000 samples: 0.4464
Validation Accuracy for 2000 samples: 0.3833
Test Accuracy for 2000 samples: 0.4467
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 

Predicted Results and Probabilities for 2000 samples (First 15 rows):
    EI_encoded Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0            1       stable       0.214768     0.420133       0.365099
1            2     decrease       0.373663     0.254343       0.371994
2            2     decrease       0.373663     0.254343       0.371994
3            2     decrease       0.373663     0.254343       0.371994
4            1       stable       0.214768     0.420133       0.365099
5            2     decrease       0.373663     0.254343       0.371994
6            1       stable       0.214768     0.420133       0.365099
7            1       stable       0.214768     0.420133       0.365099
8            1       stable       0.214768     0.420133       0.365099
9            2     decrease       0.373663     0.254

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


...

Training Accuracy for 8000 samples: 0.6166
Validation Accuracy for 8000 samples: 0.6400
Test Accuracy for 8000 samples: 0.6100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 8000 samples (First 15 rows):
    EI_encoded Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0            2       stable       0.329749     0.543725       0.126527
1            1     increase       0.170152     0.270396       0.559453
2            2       stable       0.329749     0.543725       0.126527
3            1     increase       0.170152     0.270396       0.559453
4            0     increase       0.170018     0.270176       0.559806
5            0     increase       0.170018     0.270176       0.559806
6            1     increase       0.170152     0.270396       0.559453
7            1     increase       0.170152     0.270396       0.559453
8            1     increase       0.170152     0.270396       0.559453
9            2       stable       0.329749     0.5437

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Training Accuracy for 9000 samples: 0.5054
Validation Accuracy for 9000 samples: 0.5504
Test Accuracy for 9000 samples: 0.5148
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 9000 samples (First 15 rows):
    EI_encoded Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0            0       stable       0.306623     0.375105       0.318272
1            2     increase       0.240399     0.094810       0.664791
2            1     increase       0.295122     0.205227       0.499651
3            0       stable       0.306623     0.375105       0.318272
4            0       stable       0.306623     0.375105       0.318272
5            2     increase       0.240399     0.094810       0.664791
6            2     increase       0.240399     0.094810       0.664791
7            2     increase       0.240399     0.094810       0.664791
8            0       stable       0.306623     0.375105       0.318272
9            2     increase       0.240399     0.0948

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Training Accuracy for 10000 samples: 0.4319
Validation Accuracy for 10000 samples: 0.4220
Test Accuracy for 10000 samples: 0.4207
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 10000 samples (First 15 rows):
    EI_encoded Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0            0     decrease       0.403512     0.247152       0.349336
1            0     decrease       0.403512     0.247152       0.349336
2            1     decrease       0.399375     0.237575       0.363051
3            0     decrease       0.403512     0.247152       0.349336
4            2     increase       0.379497     0.224927       0.395576
5            0     decrease       0.403512     0.247152       0.349336
6            2     increase       0.379497     0.224927       0.395576
7            2     increase       0.379497     0.224927       0.395576
8            2     increase       0.379497     0.224927       0.395576
9            0     decrease       0.403512     0.

# K-L Divergence NN Sparse Data

In [13]:
# Sample sizes to loop through
sample_sizes = range(1000, 11000, 1000)

# Prepare a list to store K-L divergence results
kl_divergence_results = []

# Loop through each sample size
for size in sample_sizes:
    print(f"\nProcessing sample size: {size}")

    # Load the combined BN data for the current sample size
    combined_data_bn = pd.read_csv(f'combined_sparse_probabilities_{size}.csv')

    # Determine features to use based on available columns
    if 'IR_State' in combined_data_bn.columns and 'EI_State' in combined_data_bn.columns:
        X = combined_data_bn[['IR_State', 'EI_State']]
    elif 'IR_State' in combined_data_bn.columns:
        X = combined_data_bn[['IR_State']]
    elif 'EI_State' in combined_data_bn.columns:
        X = combined_data_bn[['EI_State']]
    else:
        raise ValueError("No features available for training.")

    # Define labels (SP) and its probabilities
    y = combined_data_bn[['Chosen_SP_State', 'SP_Probabilities (decrease, stable, increase)']]

    # Refresh the data split for each iteration
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False, random_state=42)

    # Get the test indices
    test_indices = X_test.index

    # Get the corresponding rows from the combined BN data using the test indices
    bn_test_data = combined_data_bn.loc[test_indices]

    # Load the corresponding NN test data for the current sample size
    nn_test_data = pd.read_csv(f'test_data_nn_sparse_{size}.csv')

    # Extract NN predicted probabilities and BN ground truth probabilities
    nn_probs = nn_test_data[['Prob_decrease', 'Prob_stable', 'Prob_increase']].values
    bn_probs = bn_test_data['SP_Probabilities (decrease, stable, increase)'].apply(
        lambda x: np.array(list(map(float, x.strip('[]').split(','))))
    ).values

    # Calculate K-L divergence between NN predicted probabilities and BN ground truth probabilities
    kl_divergences = []
    output_data = []  # For tabulating output

    for i in range(len(nn_probs)):
        nn_prob = nn_probs[i]
        bn_prob = bn_probs[i]

        # Ensure both are valid probability distributions
        epsilon = 1e-10
        nn_prob = np.clip(nn_prob, epsilon, 1)
        bn_prob = np.clip(bn_prob, epsilon, 1)

        # Normalize to ensure they sum to 1
        nn_prob /= nn_prob.sum()
        bn_prob /= bn_prob.sum()

        # Compute K-L divergence
        kl_div = entropy(bn_prob, nn_prob)
        kl_divergences.append(kl_div)

        # Add data to output for tabulation
        output_data.append({
            'Sample_Index': i,
            'IR': bn_test_data.iloc[i]['IR_State'] if 'IR_State' in bn_test_data.columns else None,
            'EI': bn_test_data.iloc[i]['EI_State'] if 'EI_State' in bn_test_data.columns else None,
            'Ground_Truth_Probs': ', '.join([f'{prob:.4f}' for prob in bn_prob]),
            'NN_Probs': ', '.join([f'{prob:.4f}' for prob in nn_prob]),
            'KL_Divergence': f'{kl_div:.4f}'
        })

    # Create a DataFrame for the output data and tabulate the first few rows
    output_df = pd.DataFrame(output_data)
    print(f"\nK-L Divergence Results for {size} samples (First 5 rows):\n")
    print(tabulate(output_df.head(5), headers='keys', tablefmt='grid'))

    # Calculate and display the average K-L divergence for this sample size
    average_kl_divergence = np.mean(kl_divergences)
    std_kl_divergence = np.std(kl_divergences)
    print(f"\nAverage K-L Divergence for {size} samples: {average_kl_divergence:.4f}, Std Dev: {std_kl_divergence:.4f}")

    # Append the results to the list
    kl_divergence_results.append({
        'Sample_Size': size,
        'Average_KL_Divergence': average_kl_divergence,
        'Std_Dev': std_kl_divergence
    })

# Save the K-L divergence results to a CSV file
kl_divergence_df = pd.DataFrame(kl_divergence_results)
kl_divergence_df.to_csv('kl_div_NN_1_10_sparse.csv', index=False)

print("\nAll sample sizes have been processed and K-L divergences calculated. Results saved to 'kl_div_NN_3_30_sparse.csv'.")


Processing sample size: 1000

K-L Divergence Results for 1000 samples (First 5 rows):

+----+----------------+--------+------+------------------------+------------------------+-----------------+
|    |   Sample_Index | IR     | EI   | Ground_Truth_Probs     | NN_Probs               |   KL_Divergence |
+====+================+========+======+========================+========================+=================+
|  0 |              0 | high   | good | 0.2038, 0.0124, 0.7838 | 0.2273, 0.0590, 0.7138 |          0.0318 |
+----+----------------+--------+------+------------------------+------------------------+-----------------+
|  1 |              1 | medium | good | 0.2038, 0.0124, 0.7838 | 0.2273, 0.0590, 0.7138 |          0.0318 |
+----+----------------+--------+------+------------------------+------------------------+-----------------+
|  2 |              2 | high   | poor | 0.2999, 0.4257, 0.2745 | 0.3151, 0.3439, 0.3410 |          0.0163 |
+----+----------------+--------+------+---------